# Introduction to the Schedule of Classes API


In [5]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json

In [8]:
def get_rutgers_courses(year=2025, term=9, campus='NB'):
    """
    Fetch course data from Rutgers Schedule of Classes API
    
    Parameters:
    year (int): Academic year (default: 2025)
    term (int): Term code (default: 9 for Fall)
    campus (str): Campus code (default: 'NB' for New Brunswick)
    
    Returns:
    dict: JSON response containing course data
    """
    base_url = "https://classes.rutgers.edu/soc/api/courses.json"
    
    # Set up parameters
    params = {
        'year': year,
        'term': term,
        'campus': campus
    }
    
    try:
        # Make the API request
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        
        # Return JSON data
        return response.json()
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    
def get_open_sections(year=2025, term=9, campus='NB'):
    """
    Fetch open sections data from Rutgers Schedule of Classes API
    
    Parameters:
    year (int): Academic year (default: 2025)
    term (int): Term code (default: 9 for Fall)
    campus (str): Campus code (default: 'NB' for New Brunswick)
    
    Returns:
    dict: JSON response containing open sections data
    """
    base_url = "https://classes.rutgers.edu/soc/api/openSections.json"
    
    # Set up parameters
    params = {
        'year': year,
        'term': term,
        'campus': campus
    }
    
    try:
        # Make the API request
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        
        # Return JSON data
        return response.json()
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [11]:
# Fetch course data for Fall 2025 at New Brunswick campus
courses_data = get_rutgers_courses()
open_sections_data = get_open_sections()

# Check if data was successfully retrieved
if courses_data:
    print("Successfully retrieved course data!")
    print(f"Number of subjects: {len(courses_data)}")
    
    # Display the first few subjects to understand the data structure
    print("\nFirst 3 subjects:")
    for i, subject in enumerate(courses_data[:3]):
        print(f"{i+1}. {subject.get('courseString', 'N/A')} ({subject.get('subject', 'N/A')})")
else:
    print("Failed to retrieve course data.")
    
if open_sections_data:
    print("\nSuccessfully retrieved open sections data!")
    print(f"Number of open sections: {len(open_sections_data)}")
    
    # Display the first few open sections to understand the data structure
    print("\nFirst 3 open sections:")
    for i, section in enumerate(open_sections_data[:3]):
        # Check if section is a dictionary or string
        if isinstance(section, dict):
            print(f"{i+1}. {section.get('courseTitle', 'N/A')} - {section.get('section', 'N/A')}")
        else:
            print(f"{i+1}. {section}")  # Just print the string if it's not a dictionary

Successfully retrieved course data!
Number of subjects: 4422

First 3 subjects:
1. 01:013:111 (013)
2. 01:013:140 (013)
3. 01:013:141 (013)

Successfully retrieved open sections data!
Number of open sections: 11991

First 3 open sections:
1. 04586
2. 04588
3. 04589


## 📋 JSON Data Structure Explanation

The Rutgers Schedule of Classes API returns data in a hierarchical JSON structure. Understanding this structure is crucial for data analysis and processing.

### 🎯 Main API Endpoints

1. **`courses.json`** - Complete course catalog with detailed information
2. **`openSections.json`** - List of sections with available enrollment

---

### 📚 Courses API Structure (`courses.json`)

The response is an **array of subjects**, where each subject contains:

```json
[
  {
    "subject": "01:198",                    // Subject code
    "subjectDescription": "Byrne Seminars", // Full subject name
    "courses": [                            // Array of courses in this subject
      {
        "courseNumber": "101",              // Course number
        "title": "Introduction to...",      // Course title
        "credits": "3",                     // Credit hours
        "preReqNotes": "Prerequisites...",  // Prerequisites (if any)
        "sections": [                       // Array of course sections
          {
            "number": "01",                 // Section number
            "index": "12345",              // Registration index
            "instructors": [...],          // Array of instructor info
            "meetingTimes": [...],         // When/where classes meet
            "openStatus": true,            // Enrollment availability
            "campusCode": "NB"             // Campus location
          }
        ]
      }
    ]
  }
]
```

### 🔓 Open Sections API Structure (`openSections.json`)

Returns a **flat array of registration indices** for sections with available enrollment:

```json
[
  "12345",    // Registration index for an open section
  "12346",    // Another open section index
  "12347"     // And so on...
]
```

---

### 🔑 Key Fields Explained

| Field | Description | Example |
|-------|-------------|---------|
| `subject` | Department/subject code | "01:198", "14:332" |
| `subjectDescription` | Full department name | "Byrne Seminars", "Computer Science" |
| `courseNumber` | Course number within subject | "101", "447" |
| `title` | Course title | "Data Structures", "Calculus I" |
| `credits` | Credit hours | "3", "4", "1-3" |
| `index` | Unique registration number | "12345" |
| `openStatus` | Enrollment availability | `true`/`false` |

---

### 💡 Data Analysis Tips

- **Subject Level**: Use for department-wide analysis
- **Course Level**: Analyze course offerings and prerequisites  
- **Section Level**: Study scheduling patterns and capacity
- **Cross-Reference**: Match open sections with course details using the `index` field

## 💾 Data Download Functionality

The `download_courses_json()` function provides a convenient way to save Rutgers course data locally for offline analysis, backup, or sharing purposes.

### 🎯 What Does Data Download Do?

1. **🌐 Fetches Live Data**: Calls the Rutgers API to retrieve the most current course information
2. **💽 Local Storage**: Saves the JSON response as a formatted file on your computer
3. **📁 Smart Naming**: Auto-generates descriptive filenames (e.g., `rutgers_courses_2025_9_NB.json`)
4. **✨ Pretty Formatting**: Saves with proper indentation for human readability

### 🔧 Key Features

| Feature | Description | Benefit |
|---------|-------------|---------|
| **Auto-naming** | Generates filename with year/term/campus | Easy file organization |
| **Error Handling** | Graceful failure with helpful messages | Robust operation |
| **File Validation** | Ensures `.json` extension | Consistent file format |
| **Size Reporting** | Shows downloaded file size | Data volume awareness |

### 💡 Why Download Data?

- **🔄 Offline Analysis**: Work without internet connection
- **📊 Data Backup**: Preserve snapshots of course offerings
- **🤝 Team Sharing**: Share exact datasets with collaborators
- **⚡ Performance**: Faster repeated analysis (no API calls)
- **📈 Historical Tracking**: Compare course offerings over time

### 🛠️ Usage Scenarios

```python
# Current semester data
download_courses_json()

# Historical comparison
download_courses_json(year=2024, term=1, filename="spring_2024.json")

# Multi-campus analysis
download_courses_json(campus="NK", filename="newark_courses.json")
```

### 📋 File Contents

The downloaded JSON file contains the complete API response with all:
- Subject departments and descriptions
- Course numbers, titles, and credit information
- Section details including meeting times and instructors
- Prerequisites and course restrictions
- Enrollment capacity and availability status

In [6]:
def download_courses_json(year=2025, term=9, campus='NB', filename=None):
    """
    Download course data from Rutgers API and save it as a JSON file
    
    Parameters:
    year (int): Academic year (default: 2025)
    term (int): Term code (default: 9 for Fall)
    campus (str): Campus code (default: 'NB' for New Brunswick)
    filename (str): Custom filename (optional). If None, auto-generates filename
    
    Returns:
    str: Path to the saved file, or None if failed
    """
    # Get the course data
    courses_data = get_rutgers_courses(year, term, campus)
    
    if courses_data is None:
        print("Failed to fetch course data. Cannot download.")
        return None
    
    # Generate filename if not provided
    if filename is None:
        filename = f"rutgers_courses_{year}_{term}_{campus}.json"
    
    # Ensure filename has .json extension
    if not filename.endswith('.json'):
        filename += '.json'
    
    try:
        # Save to JSON file with pretty formatting
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(courses_data, f, indent=2, ensure_ascii=False)
        
        print(f"Successfully downloaded course data to: {filename}")
        print(f"File size: {os.path.getsize(filename) / 1024:.2f} KB")
        return filename
    
    except Exception as e:
        print(f"Error saving file: {e}")
        return None

In [7]:
# Example: Download course data to JSON file
print("Downloading course data...")

# Download with default filename
downloaded_file = download_courses_json()

# You can also specify custom parameters:
# downloaded_file = download_courses_json(year=2024, term=1, campus='NK', filename='spring_2024_newark.json')

if downloaded_file:
    print(f"\n✅ File saved successfully: {downloaded_file}")
    
    # Verify the file exists and show some info
    if os.path.exists(downloaded_file):
        with open(downloaded_file, 'r') as f:
            data = json.load(f)
        print(f"📊 File contains {len(data)} subjects")
else:
    print("❌ Download failed")

Successfully downloaded course data to: rutgers_courses_2025_9_NB.json
File size: 34042.84 KB

✅ File saved successfully: rutgers_courses_2025_9_NB.json
📊 File contains 4422 subjects
